# Download Data from Planetary Computer

[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/geoai/blob/main/docs/examples/planetary_computer.ipynb)

## Install package
To use the `geoai-py` package, ensure it is installed in your environment. Uncomment the command below if needed.

In [27]:
pip install geoai-py

## Import libraries

In [28]:
import geoai

## Retrieve collections

In [29]:
collections = geoai.pc_collection_list()
collections

Retrieved 134 collections from Planetary Computer


,id,title,description
43,3dep-lidar-classification,USGS 3DEP Lidar Classification,This collection is derived from the [USGS 3DEP...
29,3dep-lidar-copc,USGS 3DEP Lidar Point Cloud,This collection contains source data from the ...
3,3dep-lidar-dsm,USGS 3DEP Lidar Digital Surface Model,This collection is derived from the [USGS 3DEP...
44,3dep-lidar-dtm,USGS 3DEP Lidar Digital Terrain Model,This collection is derived from the [USGS 3DEP...
42,3dep-lidar-dtm-native,USGS 3DEP Lidar Digital Terrain Model (Native),This collection is derived from the [USGS 3DEP...
...,...,...,...
15,terraclimate,TerraClimate,[TerraClimate](http://www.climatologylab.org/t...
56,us-census,US Census,The [2020 Census](https://www.census.gov/progr...
87,usda-cdl,USDA Cropland Data Layers (CDLs),The Cropland Data Layer (CDL) is a product of ...
92,usgs-lcmap-conus-v13,USGS LCMAP CONUS Collection 1.3,"The [Land Change Monitoring, Assessment, and P..."


## Search NAIP imagery

In [30]:
items = geoai.pc_stac_search(
    collection="naip",
    bbox=[-76.6657, 39.2648, -76.6478, 39.2724],  # Baltimore area
    time_range="2013-01-01/2014-12-31",
)

Found 1 items matching search criteria


In [31]:
items

[<Item id=md_m_3907643_se_18_1_20130917_20131112>]

## Visualize NAIP imagery

In [32]:
geoai.pc_item_asset_list(items[0])

['image', 'metadata', 'thumbnail', 'tilejson', 'rendered_preview']

## Download NAIP imagery

In [33]:
downloaded = geoai.pc_stac_download(
    items, output_dir="data", assets=["image", "thumbnail"]
)

Processing STAC item: md_m_3907643_se_18_1_20130917_20131112
Skipping existing asset: image -> data/md_m_3907643_se_18_1_20130917_20131112_image.tif
Skipping existing asset: thumbnail -> data/md_m_3907643_se_18_1_20130917_20131112_thumbnail.jpg

Downloaded 2 assets for 1 items


## Search land cover data

In [34]:
items = geoai.pc_stac_search(
    collection="chesapeake-lc-13",
    bbox=[-76.6657, 39.2648, -76.6478, 39.2724],  # Baltimore area
    time_range="2013-01-01/2014-12-31",
    max_items=10,
)

Found 1 items matching search criteria


In [35]:
items

[<Item id=Baywide_13Class_20132014_E1640000_N1970000>]

## Visualize land cover data

In [36]:
geoai.pc_item_asset_list(items[0])

['data', 'tilejson', 'rendered_preview']

In [40]:
import planetary_computer as pc
import leafmap

lc_item = items[0] # 'items' here refers to the land cover item from previous search

# Construct the TileJSON URL for the land cover data, specifying colormap
titiler_tile_url_lc = (
    f"https://planetarycomputer.microsoft.com/api/data/v1/tilejson?"
    f"collection={lc_item.collection_id}&"
    f"item={lc_item.id}&"
    f"assets=data&"
    f"colormap_name=tab10" # Apply the colormap directly via Titiler
)

# Create a leafmap and add the tile layer
m_lc = leafmap.Map(center=(lc_item.bbox[1], lc_item.bbox[0]), zoom=12, basemap="Esri.WorldImagery")
m_lc.add_tile_layer(url=titiler_tile_url_lc, name="Land Cover Data", attribution="Planetary Computer", opacity=0.7)
m_lc

Map(center=[39.23068359277666, -76.68798051330445], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [41]:
import planetary_computer as pc
import leafmap

# Re-run NAIP search to ensure 'items_naip' contains NAIP data
items_naip = geoai.pc_stac_search(
    collection="naip",
    bbox=[-76.6657, 39.2648, -76.6478, 39.2724],  # Baltimore area
    time_range="2013-01-01/2014-12-31",
)

naip_item = items_naip[0]

# Construct the TileJSON URL directly for Planetary Computer's Data API
titiler_tile_url = (
    f"https://planetarycomputer.microsoft.com/api/data/v1/tilejson?"
    f"collection={naip_item.collection_id}&"
    f"item={naip_item.id}&"
    f"assets=image&"
    f"bands=red,green,blue"  # Explicitly request RGB bands for visualization
)

# Create a leafmap and add the tile layer
m = leafmap.Map(center=(naip_item.bbox[1], naip_item.bbox[0]), zoom=12, basemap="Esri.WorldImagery")
m.add_tile_layer(url=titiler_tile_url, name="NAIP Imagery", attribution="Planetary Computer")
m

Found 1 items matching search criteria


Map(center=[39.246357, -76.692553], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

## Download land cover data

In [42]:
geoai.pc_stac_download(items[0], output_dir="data", assets=["data", "rendered_preview"])

Processing STAC item: Baywide_13Class_20132014_E1640000_N1970000



Downloaded 2 assets for 1 items


{'Baywide_13Class_20132014_E1640000_N1970000': {'data': 'data/Baywide_13Class_20132014_E1640000_N1970000_data.tif',
  'rendered_preview': 'data/Baywide_13Class_20132014_E1640000_N1970000_rendered_preview.png'}}

In [43]:
ds = geoai.read_pc_item_asset(items[0], asset="data")

In [44]:
ds

<xarray.DataArray (band: 1, y: 10000, x: 10000)> Size: 100MB
[100000000 values with dtype=uint8]
Coordinates:
  * band         (band) int64 8B 1
  * y            (y) float64 80kB 1.98e+06 1.98e+06 ... 1.97e+06 1.97e+06
  * x            (x) float64 80kB 1.64e+06 1.64e+06 ... 1.65e+06 1.65e+06
    spatial_ref  int64 8B 0
Attributes:
    DataType:            Generic
    AREA_OR_POINT:       Area
    RepresentationType:  THEMATIC
    _FillValue:          0
    scale_factor:        1.0
    add_offset:          0.0

## Search Landsat data

In [45]:
items = geoai.pc_stac_search(
    collection="landsat-c2-l2",
    bbox=[-76.6657, 39.2648, -76.6478, 39.2724],  # Baltimore area
    time_range="2024-10-27/2024-12-31",
    query={"eo:cloud_cover": {"lt": 1}},
    max_items=10,
)

Found 1 items matching search criteria


In [46]:
items

[<Item id=LC08_L2SP_015033_20241028_02_T1>]

## Visualize Landsat data

In [47]:
geoai.pc_item_asset_list(items[0])

['qa',
 'ang',
 'red',
 'blue',
 'drad',
 'emis',
 'emsd',
 'trad',
 'urad',
 'atran',
 'cdist',
 'green',
 'nir08',
 'lwir11',
 'swir16',
 'swir22',
 'coastal',
 'mtl.txt',
 'mtl.xml',
 'mtl.json',
 'qa_pixel',
 'qa_radsat',
 'qa_aerosol',
 'tilejson',
 'rendered_preview']

In [49]:
import planetary_computer as pc
import leafmap

landsat_item = items[0]

# Construct the TileJSON URL directly for Planetary Computer's Data API
titiler_tile_url = (
    f"https://planetarycomputer.microsoft.com/api/data/v1/tilejson?"
    f"collection={landsat_item.collection_id}&"
    f"item={landsat_item.id}&"
    f"bands=red,green,blue"  # Explicitly request RGB bands for visualization
)

# Create a leafmap and add the tile layer
m_landsat = leafmap.Map(center=(landsat_item.bbox[1], landsat_item.bbox[0]), zoom=12, basemap="Esri.WorldImagery")
m_landsat.add_tile_layer(url=titiler_tile_url, name="Landsat True Color", attribution="Planetary Computer")
m_landsat

Map(center=[37.796504964993645, -78.27149033106991], controls=(ZoomControl(options=['position', 'zoom_in_text'…

In [51]:
import planetary_computer as pc
import leafmap

landsat_item = items[0] # 'items' here refers to the Landsat item from previous search

# Construct the TileJSON URL directly for Planetary Computer's Data API
titiler_tile_url_fcc = (
    f"https://planetarycomputer.microsoft.com/api/data/v1/tilejson?"
    f"collection={landsat_item.collection_id}&"
    f"item={landsat_item.id}&"
    f"bands=nir08,red,green" # Specify bands for False Color Composite
)

# Create a leafmap and add the tile layer
m_fcc = leafmap.Map(center=(landsat_item.bbox[1], landsat_item.bbox[0]), zoom=12, basemap="Esri.WorldImagery")
m_fcc.add_tile_layer(url=titiler_tile_url_fcc, name="Landsat False Color", attribution="Planetary Computer")
m_fcc

Map(center=[37.796504964993645, -78.27149033106991], controls=(ZoomControl(options=['position', 'zoom_in_text'…

In [52]:
geoai.view_pc_item(
    item=items[0],
    expression="(nir08-red)/(nir08+red)",
    rescale="-1,1",
    colormap_name="greens",
    name="NDVI Green",
)

'bbox'


## Download Landsat data

In [53]:
geoai.pc_stac_download(
    items[0], output_dir="data", assets=["nir08", "red", "green", "blue"]
)

Processing STAC item: LC08_L2SP_015033_20241028_02_T1



Downloaded 4 assets for 1 items


{'LC08_L2SP_015033_20241028_02_T1': {'red': 'data/LC08_L2SP_015033_20241028_02_T1_red.tif',
  'blue': 'data/LC08_L2SP_015033_20241028_02_T1_blue.tif',
  'green': 'data/LC08_L2SP_015033_20241028_02_T1_green.tif',
  'nir08': 'data/LC08_L2SP_015033_20241028_02_T1_nir08.tif'}}